In [ ]:
import pymupdf
# import pymupdf4llm
import pathlib
import pandas as pd
import numpy as np
from pprint import pprint
import re
from collections import Counter
from extractandclean    import extract_blocks_and_distributions , extract_heading_summary , extract_crucial_pattern_lines , filter_and_clean_gibberish
from merge              import merge_consecutive_same_font_and_left
from headerfooter       import find_alternate_page_repeats_splitv2 , find_alternate_page_repeats_split , clean_pages_of_even_odd_repeats
from tables             import extract_table_with_title
from fontstat           import generate_font_stats , get_global_font_counter , get_rare_large_fonts , map_fonts_to_heading_levels , get_significant_large_fonts , create_font_level_map , get_top_fonts , classify_font_size , extract_heading_summary
from analyzeandcompute  import unify_close_left_values , analyze_left_distribution , analyze_left_clusters , analyze_line_spacing , compute_word_count_stats , compute_line_spacing_per_page
# md_text = pymupdf4llm.to_markdown(fileinquestion)
# pathlib.Path("output.md").write_bytes(md_text.encode())
# llama_reader = pymupdf4llm.LlamaMarkdownReader()
# print(llama_reader())
# llama_docs = llama_reader.load_data(fileinquestion)

#tesseract ocr
# english
# russian
# chinese (new)
# mandarin (traditional)
# japanese
# Hindi
# Arabic
# French
# Hebrew
# German
# Korean
# Italian
# Polish
# Portugese
# Spanish 
# Indonesian3
#turkish
# Urdu
#pip install pymupdf numpy pandas sentence-transformers
#################### workflow
#parallel processing
#trigger - 


In [ ]:



###################################### something for future
# import re

# def is_gibberish(text):
#     text = text.strip()
#     if len(text) < min_length:
#         return True
#     if re.fullmatch(r'[\W_]{2,}', text):  # full of symbols
#         return True
#     alpha_count = sum(c.isalpha() for c in text)
#     return (alpha_count / len(text)) < min_alpha_ratio
# from langdetect import detect, DetectorFactory
# DetectorFactory.seed = 0

# def is_gibberish(text): #textstat or langdetect for readability or language detection
#     text = text.strip()
#     if len(text) < min_length:
#         return True
#     try:
#         lang = detect(text)
#         return False  # it's a real language
#     except:
#         return True  # detection failed = likely 
    
# from nltk.corpus import words
# from nltk.tokenize import word_tokenize
# word_set = set(words.words())

# def is_gibberish(text): # Word Tokenization and Dictionary Matching
#     tokens = word_tokenize(text.lower())
#     meaningful_words = [t for t in tokens if t in word_set]
#     if len(text.strip()) < min_length:
#         return True
#     if len(meaningful_words) / (len(tokens) + 1e-6) < 0.3:
#         return True
#     return False

# def is_gibberish(text): #Multiple Heuristics
#     text = text.strip()
#     if len(text) < min_length:
#         return True
#     if re.fullmatch(r'[\W_]{2,}', text):
#         return True
#     alpha_ratio = sum(c.isalpha() for c in text) / (len(text) + 1e-6)
#     if alpha_ratio < min_alpha_ratio:
#         return True
#     try:
#         lang = detect(text)
#         if lang not in ['en']:  # restrict to English
#             return True
#     except:
#         return True
#     return False


def is_center_aligned(left, text, page_width, font_size, tolerance=15): ######make a freaking plot on desmos if required!?!?!?!?!
    tolerance=page_width*(tolerance/100)
    avg_char_width = 0.5 * font_size  # rough estimate
    text_width = len(text) * avg_char_width
    center_text = left + text_width / 2
    center_page = page_width / 2
    return abs(center_page - center_text) #<= tolerance
# is_center_aligned(78.2, dflist[0].iloc[0]['text'], width, 14.3 )
# dflist[0]
# is_center_aligned(121.8, dflist[0].iloc[0]['text'], width, 10)

# is_center_aligned(141.3, dflist[0].iloc[0]['text'], width, 6)
# is_center_aligned(236.7, dflist[0].iloc[0]['text'], width, 14.3)

# body_font, large_fonts = get_large_fonts(font_counter, method='rms')
# page = doc[2]
# table_df, table_label = extract_table_with_title(page)

# if table_label:
#     print("Detected Table Title:", table_label["text"])
# extract_blocks_and_distributions,
# find_alternate_page_repeats_split,
# clean_pages_of_even_odd_repeats,
# merge_paragraphs_by_font


def combine_dflist_to_master_df(dflist):
    """
    Combines a list of page-level DataFrames into a single master DataFrame.
    Adds a 'page_number' column to indicate the source page for each row.
    Parameters:
        dflist (list of pd.DataFrame): A list where each DataFrame corresponds to one PDF page.
    Returns:
        pd.DataFrame: Combined DataFrame with an added 'page_number' column.
    """
    combined_rows = []
    for page_number, df in enumerate(dflist):
        if df is not None and not df.empty:
            df_copy = df.copy()
            df_copy['page_number'] = page_number
            combined_rows.append(df_copy)

    if not combined_rows:
        return pd.DataFrame()  # Return empty DataFrame if nothing valid

    master_df = pd.concat(combined_rows, ignore_index=True)
    return master_df


In [ ]:
# %%timeit
fileinquestion = "C1A/input/E0CCG5S239.pdf" #single page doc E0CCG5S239
fileinquestion = "C1A/input/TOPJUMP-PARTY-INVITATION-20161003-V01.pdf"
fileinquestion = "C1A/input/STEMPathwaysFlyer.pdf"
fileinquestion = "CustomPDFs/killer.pdf"
fileinquestion = "CustomPDFs/jess401.pdf"
fileinquestion = "C1A/input/E0CCG5S312.pdf"   #easy
fileinquestion = "C1A/input/E0H1CM114.pdf" ######### hard
doc = pymupdf.open(fileinquestion)  
page = doc[0]  # first page
width , height = page.rect.width , page.rect.height
doctoc = doc.get_toc()
docmetadata = doc.metadata
totalpage= doc.page_count
dflist   = []
mid = totalpage//2

for i in range(totalpage):
    page = doc.load_page(i)
    html = page.get_text("html")
    blocks= extract_blocks_and_distributions(html)
    dflist.append(pd.DataFrame(blocks))############### to dict if we do async
dfl = dflist
even_df, odd_df =   find_alternate_page_repeats_splitv2(dflist, mid=totalpage // 2)
dflist          =   clean_pages_of_even_odd_repeats(dflist, even_df, odd_df)
dflist          =   [filter_and_clean_gibberish(df, min_alnum_ratio=0.15, min_length=1) for df in dflist] 
averagewords = compute_word_count_stats(dflist)['global_avg_words_per_line']
font_stats_df = generate_font_stats(dflist)

dflist = unify_close_left_values(dflist, tolerance=width*0.01, min_left=0, max_left=width)

################add it back
#['page_number', 'font_size', 'font_family', 'count', 'total_words','avg_word_density']
# fontfamily_counter = Counter(font_stats_df['font_family'])
# fontsize_counter = Counter(font_stats_df['font_size'])

font_word_counts = (font_stats_df.groupby('font_size')['total_words'].sum().sort_values(ascending=False) )
# master_df = combine_dflist_to_master_df(dflist)
target_fonts = get_significant_large_fonts(font_word_counts, z_thresh=0.5)['large_fonts']
# print("Large fonts:", result['large_fonts']) #print("Z-scores:\n", result['z_scores']) #print("Body font:", result['body_font'])

spacing_list = compute_line_spacing_per_page(dflist)
all_spacings = np.concatenate(spacing_list)# If you want to concatenate all spacing into a single array:
spacing_stats = analyze_line_spacing(spacing_list)
meansp =  spacing_stats['mean_spacing']

dflist = merge_consecutive_same_font_and_left(dflist,list(font_word_counts.keys()),[meansp*0.4,meansp*1.75])
leftdf = analyze_left_distribution(dflist)['left_stats_df'] # 'left_counter': left_counter,# 'dominant_lefts': dominant_lefts,# 'left_stats_df': left_stats_df
leftdf.loc[(leftdf.left<=204) & (leftdf.percentage>=1)]

master_df = combine_dflist_to_master_df(dflist)

column_order = ['page_number',  'left',"top", 'end' ,"text",'line_height','font_size','font_family','bold','italic','word_count','word_density','parabool']
master_df[column_order]
master_df['left'] = master_df['left'].round(2)

sortmm = (master_df.loc[master_df.left<width*0.35])
unique_lefts = sorted(set(val for val in sortmm['left'].unique()))
unique_lefts




# Usage
df_summary = analyze_left_clusters([master_df], unique_lefts) #uniql are ascending
df_summary
# pprint(master_df.loc[(master_df.left==unique_lefts[0])& (master_df.word_count<=10)][['top','line_height','font_size','bold','italic','text','word_count','parabool']])
print(font_word_counts)


In [ ]:
%%timeit
fileinquestion = "C1A/input/E0CCG5S239.pdf" #single page doc E0CCG5S239
fileinquestion = "C1A/input/TOPJUMP-PARTY-INVITATION-20161003-V01.pdf"
fileinquestion = "C1A/input/STEMPathwaysFlyer.pdf"
fileinquestion = "C1A/input/E0H1CM114.pdf"
fileinquestion = "AAMine/killer.pdf"
fileinquestion = "C1A/input/E0H1CM114.pdf"
fileinquestion = "AAMine/killer.pdf"
fileinquestion = "C1A/input/E0CCG5S312.pdf"
fileinquestion = "AAMine/jess401.pdf"
fileinquestion = "C1A/input/E0CCG5S312.pdf" #single page doc E0CCG5S239 #easy


#E0CCG5S312 eazy 12 pages
#hard E0H1CM114
doc = pymupdf.open(fileinquestion)  
page = doc[0]  # first page

width = page.rect.width
height = page.rect.height
doctoc = doc.get_toc()
docmetadata = doc.metadata
totalpage= doc.page_count

dflist , repeated_all, all_font_sizes , font_counter = [], [] ,set() , Counter()
mid = totalpage//2

#if dig.dig is followed then try to use the flow...
for i in range(totalpage):
    page = doc.load_page(i)
    html = page.get_text("html")
    blocks, font_counter, line_counter = extract_blocks_and_distributions(html)
    dflist.append(pd.DataFrame(blocks))
dfl = dflist

dfcrucial = extract_crucial_pattern_lines(dflist)
dfcrucial############################################################################################################################
dfcrucial.loc[dfcrucial.bold==True]

get_top_fonts(font_counter)

even_df, odd_df =       find_alternate_page_repeats_split(dflist, mid=totalpage // 2)
dflist =                clean_pages_of_even_odd_repeats(dflist, even_df, odd_df)
dflist =        [filter_gibberish_rows(df) for df in dflist]

#mode range 
font_stats_df = generate_font_stats(dflist)
font_stats_df.sort_values(by=['page_number', 'font_size'], ascending=[True, False]).reset_index(drop=True)
font_counter = get_global_font_counter(font_stats_df)
topfonts = get_top_fonts(font_counter)
# font_level_map =  create_font_level_map(font_counter) #classify_font_size(16, font_level_map)

target_fonts = list(font_counter.keys())

#merging it now
dflist = merge_consecutive_same_font_and_left(dflist, target_fonts)
dflist = recalculate_word_stats(dflist)


font_counter
body_font, large_fonts = get_rare_large_fonts(font_counter, method='rms', freq_filter='average')

# even_df, odd_df = find_alternate_page_repeats_split(dflist, mid=totalpage // 2)
# dflist = clean_pages_of_even_odd_repeats(dflist, even_df, odd_df)
# font_level_map =  create_font_level_map(font_counter) #classify_font_size(16, font_level_map)
font_heading_map = map_fonts_to_heading_levels(large_fonts)

dflist = label_font_levels(dflist, font_heading_map) #returns the dflist
result = analyze_word_density_patterns(
    dflist,
    method='density_by_font',
    stat='median',
    threshold_factor=1.8
)

# print(f"Metric: {result['metric_name']}")
# print(f"Central Value: {result['central_value']}")
# print(f"Dense Lines: {len(result['dense_lines'])}, Sparse Lines: {len(result['sparse_lines'])}")

# for i in range(totalpage):
#     extract_table_with_title(doc[i])
dfcrucial
extract_heading_summary(dflist, levels=(['H1','H2','H3']))


In [ ]:
fileinquestion = "C1A/input/E0CCG5S312.pdf" #single page doc E0CCG5S239
#E0CCG5S312 eazy 12 pages
#hard E0H1CM114
doc = pymupdf.open(fileinquestion)  
page = doc[3]  # first page

            # “text”: (default) plain text with line breaks. No formatting, no text position details, no images.
            # “blocks”: generate a list of text blocks (= paragraphs).
            # “words”: generate a list of words (strings not containing spaces).
            # “html”: creates a full visual version of the page including any images. This can be displayed with your internet browser.
            # “dict” / “json”: same information level as HTML, but provided as a Python dictionary or resp. JSON string. See TextPage.extractDICT() for details of its structure.
            # “rawdict” / “rawjson”: a super-set of “dict” / “json”. It additionally provides character detail information like XML. See TextPage.extractRAWDICT() for details of its structure.
            # “xhtml”: text information level as the TEXT version but includes images. Can also be displayed by internet browsers.
            # “xml”: contains no images, but full position and font information down to each single text character. Use an XML module to interpret.
ab = page.get_text('xhtml')
pprint(ab)


In [ ]:
#### async

import fitz  # PyMuPDF
import pandas as pd
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor
from collections import Counter
fileinquestion = "C1A/input/E0H1CM114.pdf" ######### hard

# Dummy version of your function (replace with your real one)
def extract_blocks_and_distributions(html):
    from random import randint
    # Dummy block list
    blocks = [{"text": f"Dummy {i}", "font": "FontX", "size": randint(10, 14)} for i in range(randint(5, 15))]
    font_counter = Counter({f"FontX-{randint(10,14)}": randint(1, 5)})
    line_counter = Counter()
    return blocks, font_counter, line_counter

# ---- SEQUENTIAL VERSION ----
def process_sequential(file_path):
    doc = fitz.open(file_path)
    dflist = []
    font_counter_total = Counter()
    for i in range(doc.page_count):
        page = doc.load_page(i)
        html = page.get_text("html")
        blocks, font_counter, line_counter = extract_blocks_and_distributions(html)
        dflist.append(pd.DataFrame(blocks))
        font_counter_total.update(font_counter)
    return dflist, font_counter_total

# ---- ASYNC MULTITHREAD VERSION ----
def process_page_threaded(doc_path, page_num):
    with fitz.open(doc_path) as doc:
        page = doc.load_page(page_num)
        html = page.get_text("html")
        blocks, font_counter, line_counter = extract_blocks_and_distributions(html)
        return pd.DataFrame(blocks), font_counter

async def process_async(file_path, max_workers=4):
    loop = asyncio.get_running_loop()
    dflist = []
    font_counter_total = Counter()
    total_pages = fitz.open(file_path).page_count

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        tasks = [
            loop.run_in_executor(executor, process_page_threaded, file_path, i)
            for i in range(total_pages)
        ]
        results = await asyncio.gather(*tasks)

    for df, counter in results:
        dflist.append(df)
        font_counter_total.update(counter)

    return dflist, font_counter_total

# ---- TIMING TEST ----
fileinquestion = fileinquestion  # Change this to your file path

# Time sequential
start_seq = time.time()
dflist_seq, font_counter_seq = process_sequential(fileinquestion)
end_seq = time.time()

# Time async
start_async = time.time()
dflist_async, font_counter_async = asyncio.run(process_async(fileinquestion, max_workers=8))
end_async = time.time()

# ---- RESULT COMPARISON ----
print(f"⏱ Sequential time:      {end_seq - start_seq:.4f} seconds")
print(f"⚡ Async (Threaded) time: {end_async - start_async:.4f} seconds")

# Optional: verify correctness
print(f"\nPages (Sequential): {len(dflist_seq)}, Pages (Async): {len(dflist_async)}")


In [ ]:
dflist = []
for i in range(totalpage):
    page = doc.load_page(i)
    html = page.get_text("html")
    blocks, font_counter, line_counter = extract_blocks_and_distributions(html)
    dflist.append(pd.DataFrame(blocks))

even_df, odd_df = find_alternate_page_repeats_split(dflist, mid=totalpage // 2)
dflist = clean_pages_of_even_odd_repeats(dflist, even_df, odd_df)

def find_repeating_rows(dflist, center_page, offsets=[1, 2], tolerance=1.0):#########depriciated
    """
    Finds text lines (rows) in the center page that repeat in neighboring pages at given offsets.
    Returns a DataFrame of repeated rows likely to be headers/footers.
    """
    repeated_all = []
    df_mid = dflist[center_page]

    for offset in offsets:
        before, after = center_page - offset, center_page + offset
        if before < 0 or after >= len(dflist):
            continue

        df_before, df_after = dflist[before], dflist[after]

        for _, row in df_mid.iterrows():
            text, top = row['text'].strip(), row['top']

            def is_match(df):
                return any(
                    (abs(top - r['top']) <= tolerance) and (r['text'].strip() == text)
                    for _, r in df.iterrows()
                )

            if is_match(df_before) and is_match(df_after):
                repeated_all.append(row)

    # Drop duplicates and return as DataFrame
    return pd.DataFrame(repeated_all).drop_duplicates(subset=["text", "top", "font_size"])

repeated_rows_df = find_repeating_rows(dflist, center_page=mid)
# dflist_pages_of_repeats(dflist, repeated_rows_df)
# repeated_rows_df
# dflist
# # for i,clean in enumerate(dflist):
#     # clean.to_csv(f"analysis/hope12/f{i}.csv",index=False,encoding="utf-8")

dflister_gibberish_rows(df) for df in dflist
get_top_fonts(font_counter)
font_level_map =  create_font_level_map(font_counter) #classify_font_size(16, font_level_map)
dflist_font_levels(dflist)
extract_heading_summary(dflist, levels=(['H1','H2']))
font_counter
i=0
# repeated_rows_df
dflist[1]
font_level_map
extract_heading_summary(dflist, levels=(['H1','H2']))


In [ ]:
fileinquestion = "C1A/input/E0H1CM114.pdf"
#E0CCG5S312 eazy 12 pages
doc = pymupdf.open(fileinquestion)  
doctoc = doc.get_toc()
docmetadata = doc.metadata
totalpage= doc.page_count

dflist , repeated_all, all_font_sizes , font_counter = [], [] ,set() , Counter()
mid = totalpage//2

#if dig.dig is followed then try to use the flow...

for i in range(totalpage):
    page = doc.load_page(i)
    html = page.get_text("html")
    blocks, font_counter, line_counter = extract_blocks_and_distributions(html)
    dflist.append(pd.DataFrame(blocks))

even_df, odd_df = find_alternate_page_repeats_split(dflist, mid=totalpage // 2)
dflist_pages_of_even_odd_repeats(dflist, even_df, odd_df)

paragraphs = []
for i, df in enumerate(dflist
    merged = merge_paragraphs_by_font(df, font_min=8.0, font_max=14.0)  # You can define ranges
    for para in merged:
        para['page'] = i
    paragraphs.extend(para)

paragraph_df = pd.DataFrame(paragraphs)
merged = merge_paragraphs_by_font(df, font_min=8.0, font_max=14.0, font_tolerance=1.0, line_gap=3.0)

# repeated_rows_df = find_repeating_rows(dflist, center_page=mid)
# dflist_pages_of_repeats(dflist, repeated_rows_df)
# repeated_rows_df
# dflist
# # for i,clean in enumerate(dflist
#     # clean.to_csv(f"analysis/hope12/f{i}.csv",index=False,encoding="utf-8")

dflister_gibberish_rows(df) for df in dflist
get_top_fonts(font_counter)
font_level_map =  create_font_level_map(font_counter) #classify_font_size(16, font_level_map)
dflist = label_font_levels(dflist, font_level_map)
extract_heading_summary(dflist=(['H1','H2']))



In [ ]:
for i, page_df in enumerate(dflist
    paras = merge_paragraphs_by_font(page_df)
    for p in paras:
        p['page'] = i  # Add page number
    merged_paras.extend(paras)

# Convert to DataFrame for analysis
paragraph_df = pd.DataFrame(merged_paras)


In [ ]:
#############EXP
tablezdf
for i in range(totalpage):
    page = doc[i]
    table_df, title = extract_table_with_title(page)

    if table_df is not None:
        print(f"\nTitle: {title}\n")
        print(table_df.head())
        tabs


In [ ]:
#GRAVEYARD

# page = doc[1] #not index, its page
# for page in reversed(doc):
# for page in doc.pages(start, stop, step):
locationn= "analysis/hope1"
alldfs,thetexts , linkstotal , linkstotalnext , annotstotal , html = [] , [] , [] , [] , [] , []
def dumpshit(doc):
    for i,page in enumerate(doc): # iterate the document pages
        text = page.get_text() # get plain text encoded as UTF-8
        thetexts.append(text)
        links = page.get_links()
        linkstotal.append(links)
        link = page.first_link  # a `Link` object or `None`
        linktemp =[]
        while link: 
            link = link.next # get next link, last one has `None` in its `next`
            linktemp.append(link)
        linkstotalnext.append(linktemp)
        # for annot in page.annots():
            # print(f'Annotation on page: {page.number} with type: {annot.type} and rect: {annot.rect}')
        # for field in page.widgets():
            # print(f'Widget on page: {page.number} with type: {field.type} and rect: {field.rect}')
        text = page.get_text("html")
            # Use one of the following strings for opt to obtain different formats [2]:
            # “text”: (default) plain text with line breaks. No formatting, no text position details, no images.
            # “blocks”: generate a list of text blocks (= paragraphs).
            # “words”: generate a list of words (strings not containing spaces).
            # “html”: creates a full visual version of the page including any images. This can be displayed with your internet browser.
            # “dict” / “json”: same information level as HTML, but provided as a Python dictionary or resp. JSON string. See TextPage.extractDICT() for details of its structure.
            # “rawdict” / “rawjson”: a super-set of “dict” / “json”. It additionally provides character detail information like XML. See TextPage.extractRAWDICT() for details of its structure.
            # “xhtml”: text information level as the TEXT version but includes images. Can also be displayed by internet browsers.
            # “xml”: contains no images, but full position and font information down to each single text character. Use an XML module to interpret.
        # with open("output.html", "w") as f:
        text = re.sub(r'<img[^>]*>', '<image>', text)
        html.append(text)
        with open(f"analysis/hope1/f{i}.html",'w',encoding="utf-8") as f:
            f.write(text)
 
        # Font
        # Alignment
        # cases
        #indentation, new pages, 
        # 1 1.0
        #toc/index - 
        # blocks of code
        # 
# for i in range(12)
page= doc[0]
text = page.get_text("html")
# pprint(text)
print(doc[1].get_text("blocks") == doc[1].get_text("blocks"))
htmx=(doc[0].get_text("html"))
with open ("html.html", "w") as f:
    f.write(htmx)
htmx

In [ ]:
#Depreciated 
def clean_pages_of_even_odd_repeats(dflist, even_df, odd_df, tolerance=1.0):
    """
    Cleans even-indexed pages using even_df and odd-indexed pages using odd_df.
    If even_df == odd_df, treats all pages the same.
    """
    cleaned_pages = []
    # Check if both are the same
    same_repeats = ( set((r['text'].strip(), round(r['top'], 1)) for _, r in even_df.iterrows()) == set((r['text'].strip(), round(r['top'], 1)) for _, r in odd_df.iterrows()) )
    for idx, df in enumerate(dflist):
        if same_repeats:
            compare_df = even_df  # or odd_df, doesn't matter
        else:
            compare_df = even_df if idx % 2 == 0 else odd_df
        mask = df.apply(
            lambda row: not any(
                (abs(row['top'] - rep['top']) <= tolerance)
                and (row['text'].strip() == rep['text'].strip())
                for _, rep in compare_df.iterrows()
            ),
            axis=1
        )
        cleaned_df = df[mask].reset_index(drop=True)
        cleaned_pages.append(cleaned_df)
    return cleaned_pages

def clean_pages_of_repeats(dflist, repeated_df, tolerance=1.0): ##################depreciated
    """
    Removes rows from each page in dflist that match (in position and text) any row in repeated_df.
    Useful for cleaning common headers/footers.
    """
    cleaned_pages = []

    for df in dflist:
        mask = df.apply(
            lambda row: not any(
                (abs(row['top'] - rep['top']) <= tolerance) and (row['text'].strip() == rep['text'].strip())
                for _, rep in repeated_df.iterrows()
            ),
            axis=1
        )
        cleaned_df = df[mask].reset_index(drop=True)
        cleaned_pages.append(cleaned_df)

    return cleaned_pages


def find_repeating_rows(dflist, center_page, offsets=[1, 2], tolerance=1.0):#########depriciated
    """
    Finds text lines (rows) in the center page that repeat in neighboring pages at given offsets.
    Returns a DataFrame of repeated rows likely to be headers/footers.
    """
    repeated_all = []
    df_mid = dflist[center_page]

    for offset in offsets:
        before, after = center_page - offset, center_page + offset
        if before < 0 or after >= len(dflist):
            continue

        df_before, df_after = dflist[before], dflist[after]

        for _, row in df_mid.iterrows():
            text, top = row['text'].strip(), row['top']

            def is_match(df):
                return any(
                    (abs(top - r['top']) <= tolerance) and (r['text'].strip() == text)
                    for _, r in df.iterrows()
                )

            if is_match(df_before) and is_match(df_after):
                repeated_all.append(row)

    # Drop duplicates and return as DataFrame
    return pd.DataFrame(repeated_all).drop_duplicates(subset=["text", "top", "font_size"])

def find_alternate_page_repeats_splitold(dflist, mid):
    def get_matches(df_base, df1, df2):
        matched = []
        for _, row in df_base.iterrows():
            text = row['text'].strip()
            top = row['top']

            def match(df):
                return any(
                    (abs(top - r['top']) <= 1.0) and (r['text'].strip() == text)
                    for _, r in df.iterrows()
                )
            if match(df1) and match(df2):
                matched.append(row)
        return pd.DataFrame(matched)
    even_matches, odd_matches = pd.DataFrame(), pd.DataFrame()

    # Mid page even
    if mid % 2 == 0:
        page_even = dflist[mid]
        before_even = dflist[mid - 2] if mid - 2 >= 0 else None
        after_even = dflist[mid + 2] if mid + 2 < len(dflist) else None

        if before_even is not None and after_even is not None:
            even_matches = get_matches(page_even, before_even, after_even)
        elif before_even is not None:
            even_matches = get_matches(page_even, before_even)
        elif after_even is not None:
            even_matches = get_matches(page_even, after_even)

    # Mid - 1 page odd
    if mid - 1 >= 0 and (mid - 1) % 2 == 1:
        odd_page_idx = mid - 1
        page_odd = dflist[odd_page_idx]
        before_odd = dflist[odd_page_idx - 2] if odd_page_idx - 2 >= 0 else None
        after_odd = dflist[odd_page_idx + 2] if odd_page_idx + 2 < len(dflist) else None

        if before_odd is not None and after_odd is not None:
            odd_matches = get_matches(page_odd, before_odd, after_odd)
        elif before_odd is not None:
            odd_matches = get_matches(page_odd, before_odd)
        elif after_odd is not None:
            odd_matches = get_matches(page_odd, after_odd)
    return even_matches, odd_matches

def merge_paragraphs_by_font0(df, font_tolerance=1, line_gap=2.0):
    """
    Groups consecutive lines with similar font size and small vertical gap into paragraphs.

    Parameters:
        df (pd.DataFrame): A single-page DataFrame containing 'top', 'font_size', 'text' columns.
        font_tolerance (float): Allowed deviation in font size to consider lines as same style.
        line_gap (float): Max vertical gap between lines to be considered part of the same paragraph.

    Returns:
        List of dicts: Each dict represents a merged paragraph block.
    """
    if df.empty:
        return []

    # Sort by vertical position
    # df_sorted = df.sort_values(by='top').reset_index(drop=True)
    df_sorted = df
    paragraphs = []
    current_para = {
        'top': df_sorted.loc[0, 'top'],
        'font_size': df_sorted.loc[0, 'font_size'],
        'text': df_sorted.loc[0, 'text']
    }

    for i in range(1, len(df_sorted)):
        prev = df_sorted.loc[i - 1]
        curr = df_sorted.loc[i]

        same_font = (abs(curr['font_size'] - prev['font_size']) <= font_tolerance)
        # close_enough = abs(curr['top'] - prev['top']) <= (prev['line_height'] + line_gap)

        if same_font : #and close_enough:
            current_para['text'] += ' ' + curr['text']
        else:
            paragraphs.append(current_para)
            current_para = {
                'top': curr['top'],
                'font_size': curr['font_size'],
                'text': curr['text']
            }

    paragraphs.append(current_para)
    return 


def merge_consecutive_same_font_and_leftoldnew(dflist, target_fonts):
    """
    Merges lines in-place within each DataFrame in dflist where:
    - font_size ∈ target_fonts
    - consecutive lines share same font_size and left
    Adds a 'parabool' column to mark whether a line is a merged paragraph.
    
    Parameters:
        dflist (list of pd.DataFrame): Each DataFrame is a page.
        target_fonts (list): List of font sizes to consider for paragraph merging.
    
    Returns:
        list of pd.DataFrame: Modified DataFrames with merged paragraphs and 'parabool' flag.
    """
    updated_dflist = []

    for df in dflist:
        if df.empty or 'left' not in df.columns or 'font_size' not in df.columns:
            df['parabool'] = False
            updated_dflist.append(df)
            continue

        df = df.copy()
        df['parabool'] = False
        drop_indices = set()

        for left_value in sorted(df['left'].unique()):
            # Get indices where left matches
            group_df = df[(df['left'] == left_value) & (df['font_size'].isin(target_fonts))]
            indices = group_df.index.tolist()
            i = 0

            while i < len(indices):
                current_idx = indices[i]
                current_font = df.loc[current_idx, 'font_size']
                current_text = df.loc[current_idx, 'text']

                merged = False
                j = i + 1

                while j < len(indices):
                    next_idx = indices[j]
                    next_font = df.loc[next_idx, 'font_size']

                    if next_font == current_font:
                        # Merge text
                        df.at[current_idx, 'text'] += ' ' + df.loc[next_idx, 'text']
                        drop_indices.add(next_idx)
                        merged = True
                        j += 1
                    else:
                        break

                if merged:
                    df.at[current_idx, 'parabool'] = True
                else:
                    df.at[current_idx, 'parabool'] = False

                i = j  # skip merged rows

        # Drop merged lines
        df = df.drop(index=list(drop_indices)).reset_index(drop=True)
        updated_dflist.append(df)

    return updated_dflist


def extract_table_with_titleold(page, max_title_distance=50): #################Old
    """
    Extracts the first table and its potential title from a PyMuPDF page.

    Parameters:
        page (fitz.Page): The PDF page object.
        max_title_distance (float): Max vertical distance (in pt) to search above the table for a title.

    Returns:
        tuple:
            - pd.DataFrame: The extracted table (if found), else None.
            - str: The possible table title (if found), else None.
    """
    tables = page.find_tables()
    print(f"{len(tables.tables)} table(s) found.")

    if not tables.tables:
        return None, None

    table = tables[0]
    table_data = table.extract()
    table_df = pd.DataFrame(table_data)

    print("First table extracted:")
    pprint(table_data)

    # Look for text just above the table
    bbox = table.bbox
    text_dict = page.get_text("dict")
    possible_titles = []

    for block in text_dict.get("blocks", []):
        if "lines" in block and block["bbox"][3] < bbox[1]:  # Above the table
            if abs(block["bbox"][3] - bbox[1]) <= max_title_distance:
                text = " ".join(
                    span["text"] for line in block["lines"] for span in line["spans"]
                ).strip()
                if text:
                    possible_titles.append((block["bbox"][1], text))

    # Closest first
    possible_titles.sort(key=lambda x: -x[0])

    if possible_titles:
        title = possible_titles[0][1]
        print("Possible table title:", title)
    else:
        title = None
        print("No clear title found above the table.")

    return table_df, title


def filter_and_clean_gibberish(df, text_column='text', min_alnum_ratio=0.1, min_length=1):
    """
    Replaces gibberish parts of the text with spaces. If the entire string is gibberish,
    the row is removed.
    Replaces gibberish parts of the text with spaces. Keeps tokens like '2.1', 'a.', '1.' etc.
    Removes purely symbolic tokens. Drops rows if nothing meaningful remains.

    Parameters:
        df (pd.DataFrame): The input DataFrame with a text column.
        text_column (str): Column name that contains the text.
        min_alnum_ratio (float): Minimum ratio of alphanumeric chars to keep a token.
        min_length (int): Minimum total length of cleaned string to keep the row.

    Returns:
        pd.DataFrame: Cleaned DataFrame with gibberish removed or cleaned.
    """

    def is_meaningful(token):
        token = token.strip()
        if not token:
            return False

        # Keep tokens like: 2.1, a., 1., v2.0, etc.
        if re.match(r'^[a-zA-Z0-9]+\.$', token):     # a. or 1.
            return True
        if re.match(r'^[a-zA-Z]?[0-9]*\.[0-9]+$', token):  # 2.1, 0.3, v2.0
            return True

        alnum_ratio = sum(c.isalnum() for c in token) / (len(token) + 1e-6)
        return alnum_ratio >= min_alnum_ratio

    def clean_text(text):
        text = str(text).strip()
        if len(text) < min_length:
            return None

        # Split tokens but keep punctuation and symbols separated
        tokens = re.split(r'(\W+)', text)
        cleaned_tokens = []

        for token in tokens:
            if is_meaningful(token):
                cleaned_tokens.append(token)
            else:
                cleaned_tokens.append(' ')

        cleaned = ''.join(cleaned_tokens).strip()
        if len(cleaned) < min_length or all(c.isspace() for c in cleaned):
            return None
        return cleaned

    df[text_column] = df[text_column].apply(clean_text)
    df = df[df[text_column].notna()].reset_index(drop=True)
    return df


def merge_consecutive_same_font_and_leftv44(dflist, target_fonts, acceptable_spacing_range=[0.0, 2.0], min_words_required=3):
    min_spacing, max_spacing = acceptable_spacing_range
    updated_dflist = []

    for df in dflist:
        if df.empty or 'left' not in df.columns or 'font_size' not in df.columns:
            df['parabool'] = False
            updated_dflist.append(df)
            continue

        df = df.copy()
        df['parabool'] = False
        drop_indices = set()

        for left_value in sorted(df['left'].unique()):
            group_df = df[(df['left'] == left_value) & (df['font_size'].isin(target_fonts))]
            indices = group_df.index.tolist()
            i = 0

            while i < len(indices):
                current_idx = indices[i]
                current_font = df.loc[current_idx, 'font_size']
                current_bold = df.loc[current_idx, 'bold']
                current_italic = df.loc[current_idx, 'italic']
                current_top = df.loc[current_idx, 'top']
                current_word_count = df.loc[current_idx, 'word_count']

                j = i + 1
                merged = False

                while j < len(indices):
                    next_idx = indices[j]
                    next_font = df.loc[next_idx, 'font_size']
                    next_bold = df.loc[next_idx, 'bold']
                    next_italic = df.loc[next_idx, 'italic']
                    next_top = df.loc[next_idx, 'top']

                    if (
                        next_font != current_font or
                        (current_word_count > min_words_required and (
                            next_bold != current_bold or
                            next_italic != current_italic))
                    ):
                        break

                    top_diff = abs(next_top - current_top)
                    if not (min_spacing <= top_diff <= max_spacing):
                        break

                    df.at[current_idx, 'text'] += ' ' + df.loc[next_idx, 'text']
                    df.at[current_idx, 'word_count'] += df.loc[next_idx, 'word_count']
                    drop_indices.add(next_idx)
                    merged = True
                    j += 1
                    current_top = next_top
                    current_word_count = df.at[current_idx, 'word_count']

                df.at[current_idx, 'parabool'] = merged
                i = j

        df = df.drop(index=list(drop_indices)).reset_index(drop=True)
        updated_dflist.append(df)

    return updated_dflist


def merge_paragraphs_by_font(df, font_min=8.0, font_max=14.0, font_tolerance=1.0,line_gap=2.0): #maybe add a center align criteria
    """
    Groups consecutive lines (in original order) with similar font size and small vertical gap into paragraphs.
    No sorting is applied — assumes df is already in reading order.

    Parameters:
        df (pd.DataFrame): Input DataFrame with 'top', 'font_size', 'line_height', 'text' columns.
        font_min (float): Minimum font size to allow.
        font_max (float): Maximum font size to allow.
        font_tolerance (float): Allowed deviation in font size between lines.
        line_gap (float): Maximum allowed vertical gap to group lines.

    Returns:
        List[dict]: List of merged paragraph dictionaries.
    """
    if df.empty:
        return []
    # Filter font range
    df = df[(df['font_size'] >= font_min) & (df['font_size'] <= font_max)].reset_index(drop=True)
    if df.empty:
        return []

    paragraphs = []
    current_para = {
        'top': df.loc[0, 'top'],
        'font_size': df.loc[0, 'font_size'],
        'text': df.loc[0, 'text'],
        'line_count': 1
    }

    for i in range(1, len(df)):
        prev = df.loc[i - 1]
        curr = df.loc[i]
    
        same_font = abs(curr['font_size'] - prev['font_size']) <= font_tolerance
        small_gap = abs(curr['top'] - prev['top']) <= (prev['line_height'] + line_gap)

        if same_font and small_gap:
            current_para['text'] += ' ' + curr['text']
            current_para['line_count'] += 1
        else:
            paragraphs.append(current_para)
            current_para = {
                'top': curr['top'],
                'font_size': curr['font_size'],
                'text': curr['text'],
                'line_count': 1
            }

    paragraphs.append(current_para)
    return paragraphs

def filter_gibberish_rows(df, text_column='text', min_alpha_ratio=0.1, min_length=1):
    """
    Removes rows from df where the text is mostly non-alphabetic (e.g., --------, ...., ====),
    or is too short to be meaningful.

    Parameters:
        df (pd.DataFrame): The input DataFrame with a text column.
        text_column (str): Column name that contains the text.
        min_alpha_ratio (float): Minimum ratio of alphabetic chars to keep the row.
        min_length (int): Minimum total length of string to keep it.

    Returns:
        pd.DataFrame: Cleaned DataFrame with gibberish lines removed.
    """
    def is_gibberish(text):
        if len(text.strip()) < min_length:
            return True
        alpha_count = sum(c.isalpha() for c in text)
        return (alpha_count / len(text)) < min_alpha_ratio

    mask = df[text_column].apply(lambda t: not is_gibberish(t))
    return df[mask].reset_index(drop=True)



def extract_table_with_titleold(page, max_title_distance=50): #################Old
    """
    Extracts the first table and its potential title from a PyMuPDF page.

    Parameters:
        page (fitz.Page): The PDF page object.
        max_title_distance (float): Max vertical distance (in pt) to search above the table for a title.

    Returns:
        tuple:
            - pd.DataFrame: The extracted table (if found), else None.
            - str: The possible table title (if found), else None.
    """
    tables = page.find_tables()
    print(f"{len(tables.tables)} table(s) found.")

    if not tables.tables:
        return None, None

    table = tables[0]
    table_data = table.extract()
    table_df = pd.DataFrame(table_data)

    print("First table extracted:")
    pprint(table_data)

    # Look for text just above the table
    bbox = table.bbox
    text_dict = page.get_text("dict")
    possible_titles = []

    for block in text_dict.get("blocks", []):
        if "lines" in block and block["bbox"][3] < bbox[1]:  # Above the table
            if abs(block["bbox"][3] - bbox[1]) <= max_title_distance:
                text = " ".join(
                    span["text"] for line in block["lines"] for span in line["spans"]
                ).strip()
                if text:
                    possible_titles.append((block["bbox"][1], text))

    # Closest first
    possible_titles.sort(key=lambda x: -x[0])

    if possible_titles:
        title = possible_titles[0][1]
        print("Possible table title:", title)
    else:
        title = None
        print("No clear title found above the table.")

    return table_df, title



def merge_consecutive_same_font_and_leftv2(dflist, target_fonts):
    """
    Merges lines within each DataFrame in dflist where:
    - font_size ∈ target_fonts
    - consecutive lines share same font_size, left alignment, and bold/italic state
    Adds a 'parabool' column to mark whether a line is a merged paragraph.

    Parameters:
        dflist (list of pd.DataFrame): Each DataFrame is a page.
        target_fonts (list): List of font sizes to consider for paragraph merging.

    Returns:
        list of pd.DataFrame: Modified DataFrames with merged paragraphs and 'parabool' flag.
    """
    updated_dflist = []

    for df in dflist:
        if df.empty or 'left' not in df.columns or 'font_size' not in df.columns:
            df['parabool'] = False
            updated_dflist.append(df)
            continue

        df = df.copy()
        df['parabool'] = False
        drop_indices = set()

        for left_value in sorted(df['left'].unique()):
            group_df = df[(df['left'] == left_value) & (df['font_size'].isin(target_fonts))]
            indices = group_df.index.tolist()
            i = 0

            while i < len(indices):
                current_idx = indices[i]
                current_font = df.loc[current_idx, 'font_size']
                current_bold = df.loc[current_idx, 'bold']
                current_italic = df.loc[current_idx, 'italic']

                j = i + 1
                merged = False

                while j < len(indices):
                    next_idx = indices[j]
                    next_font = df.loc[next_idx, 'font_size']
                    next_bold = df.loc[next_idx, 'bold']
                    next_italic = df.loc[next_idx, 'italic']

                    # Stop if font or bold/italic style mismatch
                    if (
                        next_font != current_font or
                        next_bold != current_bold or
                        next_italic != current_italic
                    ):
                        break

                    # Merge text
                    df.at[current_idx, 'text'] += ' ' + df.loc[next_idx, 'text']
                    drop_indices.add(next_idx)
                    merged = True
                    j += 1

                df.at[current_idx, 'parabool'] = merged
                i = j

        df = df.drop(index=list(drop_indices)).reset_index(drop=True)
        updated_dflist.append(df)

    return updated_dflist


def merge_consecutive_same_font_and_leftv3(dflist, target_fonts, acceptable_spacing_range=[0.0, 2.0]):
    """
    Merges lines within each DataFrame in dflist where:
    - font_size ∈ target_fonts
    - consecutive lines share same font_size, left alignment, and bold/italic state
    - vertical spacing (top difference) is within acceptable_spacing_range

    Adds:
    - 'parabool' column: True if line is a merged paragraph.
    - 'end' column: set to its own top if no merge, or the last merged line's top.

    Parameters:
        dflist (list of pd.DataFrame): Each DataFrame is a page.
        target_fonts (list): Font sizes to consider for merging.
        acceptable_spacing_range (list): [min_spacing, max_spacing] range to allow merging lines.

    Returns:
        list of pd.DataFrame: Modified DataFrames with merged paragraphs and 'parabool'/'end' flags.
    """
    min_spacing, max_spacing = acceptable_spacing_range
    updated_dflist = []

    for df in dflist:
        if df.empty or 'left' not in df.columns or 'font_size' not in df.columns:
            df['parabool'] = False
            df['end'] = df['top'] if 'top' in df.columns else 0
            updated_dflist.append(df)
            continue

        df = df.copy()
        df['parabool'] = False
        df['end'] = df['top']
        drop_indices = set()

        for left_value in sorted(df['left'].unique()):
            group_df = df[(df['left'] == left_value) & (df['font_size'].isin(target_fonts))]
            indices = group_df.index.tolist()
            i = 0

            while i < len(indices):
                current_idx = indices[i]
                current_font = df.loc[current_idx, 'font_size']
                current_bold = df.loc[current_idx, 'bold']
                current_italic = df.loc[current_idx, 'italic']
                current_top = df.loc[current_idx, 'top']
                final_end = current_top

                j = i + 1
                merged = False

                while j < len(indices):
                    next_idx = indices[j]
                    next_font = df.loc[next_idx, 'font_size']
                    next_bold = df.loc[next_idx, 'bold']
                    next_italic = df.loc[next_idx, 'italic']
                    next_top = df.loc[next_idx, 'top']

                    # Stop if font or style mismatch
                    if (
                        next_font != current_font or
                        next_bold != current_bold or
                        next_italic != current_italic
                    ):
                        break

                    # Check top spacing constraint
                    top_diff = abs(next_top - current_top)
                    if not (min_spacing <= top_diff <= max_spacing):
                        break

                    # Merge text
                    df.at[current_idx, 'text'] += ' ' + df.loc[next_idx, 'text']
                    final_end = next_top
                    drop_indices.add(next_idx)
                    merged = True
                    j += 1
                    current_top = next_top  # update for cascading merge

                df.at[current_idx, 'parabool'] = merged
                df.at[current_idx, 'end'] = final_end
                i = j

        df = df.drop(index=list(drop_indices)).reset_index(drop=True)
        updated_dflist.append(df)

    return updated_dflist


#spandan

def merge_consecutive_same_font_and_leftv0(dflist, target_fonts):
    """
    Merges lines in-place within each DataFrame in dflist where:
    - font_size ∈ target_fonts
    - consecutive lines share same font_size and left
    - lines are NOT bold or italic
    Adds a 'parabool' column to mark whether a line is a merged paragraph.

    Parameters:
        dflist (list of pd.DataFrame): Each DataFrame is a page.
        target_fonts (list): List of font sizes to consider for paragraph merging.

    Returns:
        list of pd.DataFrame: Modified DataFrames with merged paragraphs and 'parabool' flag.
    """
    updated_dflist = []

    for df in dflist:
        if df.empty or 'left' not in df.columns or 'font_size' not in df.columns:
            df['parabool'] = False
            updated_dflist.append(df)
            continue

        df = df.copy()
        df['parabool'] = False
        drop_indices = set()

        for left_value in sorted(df['left'].unique()):
            group_df = df[(df['left'] == left_value) & (df['font_size'].isin(target_fonts))]
            indices = group_df.index.tolist()
            i = 0

            while i < len(indices):
                current_idx = indices[i]
                current_font = df.loc[current_idx, 'font_size']

                # Skip bold/italic rows
                if df.loc[current_idx, 'bold'] : #or df.loc[current_idx, 'italic']: #ignore the italics for now
                    df.at[current_idx, 'parabool'] = False
                    i += 1
                    continue

                j = i + 1
                merged = False

                while j < len(indices):
                    next_idx = indices[j]
                    next_font = df.loc[next_idx, 'font_size']

                    # Stop if font differs or next is bold/italic
                    if next_font != current_font or df.loc[next_idx, 'bold'] or df.loc[next_idx, 'italic']:
                        break

                    # Merge
                    df.at[current_idx, 'text'] += ' ' + df.loc[next_idx, 'text']
                    drop_indices.add(next_idx)
                    merged = True
                    j += 1

                df.at[current_idx, 'parabool'] = merged
                i = j

        # Drop merged rows
        df = df.drop(index=list(drop_indices)).reset_index(drop=True)
        updated_dflist.append(df)

    return updated_dflist

In [ ]:
page = doc[1]
print(page.get_text('html'))
